<a href="https://colab.research.google.com/github/talhavawda/tic-tac-toe-monte-carlo/blob/master/TicTacToe_MonteCarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tic Tac Toe using Monte Carlo Tree Search

 ## COMP703 (ARTIFICIAL INTELLIGENCE) 2021 ASSIGNMENT 1
 
 Team: A1GroupWC

 Team Members:
 -  Azhar Mohamed&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(218006491)
 - Yashlin Naidu &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(216019492)
 - Ricardo Pillay &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(218009114)
 - Ahmad Jawaad Shah&ensp;&ensp;&ensp;(218029400)
 - Talha Vawda &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;(218023210)

<br>

## Acknowledgements and References
- AIMA
- COMP703 Lectures and Notes
- https://www.python.org/dev/peps/pep-0008/ (PEP 8 -- Style Guide for Python Code)
- Python: check if list is empty (https://stackoverflow.com/questions/53513/how-do-i-check-if-a-list-is-empty)



# Introduction and Background
Talk about game variant 


# Imports and constants
This cell contains all the libraries that is needed to run this game. Furthermore, constant initializations are done here and will be used throughout the notebook

Please run all cells in order (on your first run) so that classes that  are used later on are 'compiled' and accessible/existing before they are used

In [3]:
"""
    Imports
"""
import numpy as np
import math
import copy
import random
 
 
"""
    Constants
"""
# 'C' is used in the UCT formula
# Theoretically, C should be set to sqrt(2) but play around with different values to see which works best
C = 2

# Working with tuples
listTuples = list()
listTuples.append((0,1))
listTuples.append((0,2))
listTuples.append((0,3))
print(listTuples[0])
print(listTuples[0][0], "\t", listTuples[0][1])
print(listTuples[1][0], "\t", listTuples[1][1])
print(listTuples[2][0], "\t", listTuples[2][1])

if listTuples:
    print("not empty")
else:
    print("empty")

(0, 1)
0 	 1
0 	 2
0 	 3
not empty


# State Class

Represents a State of the Tic Tac Toe game


In [4]:
class State:
    """
        A Tic Tac Toe state for the Monte Carlo Tree Search Algorithm
        A State of the game is a specific board scenario, along with the player whose turn it is to play next
    """


    def __init__(self, board=None, turn=None):
        """
            Constructor for the initial creation of a Tic Tac Toe State.
    
        """

        if board != None:
            # Create a board for an existing game state
            self.board = board
        else:
            # Create the board for the initial game state
            self.board = TicTacToeGame.initial_game_board()
            self.turn = TicTacToeGame.X # this variable represents the player whose turn it is to play on this current state (current player) | X plays first on an initial board

        # DEPRECATED
        #self.is_terminal_state = False
        #self.winner = TIC_TAC_TOE_GAME.IN_PROGRESS 
        #self.check_game_over() # This function updates self.is_terminal_state and self.winner 



        if turn != None:
            self.turn = turn # The player whose turn it is to currently play on this current state, either [Deprecated: "X" or "O"] TicTacToeGame.X or TicTacToeGame.O
        else:
            #Todo - determine whose turn it is by counting the number of X's and O's on the board. if X's==O's then X's turn otherwise O's
            num_x = (np.array(self.board) == TicTacToeGame.X).sum()
            num_y = (np.array(self.board) == TicTacToeGame.O).sum()
            if num_x > num_y:
              self.turn = TicTacToeGame.O
            else:
              self.turn = TicTacToeGame.X

        
    def toggle_turn(self, turn):
        """
            ALT:
            if turn == TicTacToeGame.X:
                return TicTacToeGame.O
             else:
                return = TicTacToeGame.O
        """

        return -turn # since TicTacToeGame.X == -TicTacToeGame.O and vice versa
        

# TicTacToeGame class

In [15]:
class TicTacToeGame:
    """ 
        A class that represents our variant of the Tic Tac Toe game
        The purpose is to initialise a Tic Tac Toe game by creating a nxn square board
        and placing the neutral tokens on it, and to hold the n, k, and neutral_tokens values


        The board is being implemented as a nxn array with an index pair representing
        a square on the board and the element values being either [Deprecated: 'S', 'X', 'O', ""]
        2, 1, -1, 0 to represent a neutral token, the X token, the O token, and an empty square (where
        no token has been played yet)

        We are having a specific game class as we are demonstrating various examples of Tic Tac Toe game scenarios
        where different games could have a different size board, so we're going to need different game instances
        representing different games with their boards to pass into the MCTS algorithm

        An instance of this class (representing a game scenario) will be passed into the Monte Carlo Tree Search (MCTS) algorithm
    """

    # Constants of this Tic Tac Toe Game class
    X = 1  # Player/Token X
    EMPTY = 0  # An empty square (no tokens placed on it yet)
    O = -1  # Player/Token O
    S = 2  # Neutral Token
    IN_PROGRESS = 3  # Game in progress
    DRAW = 4  # Game over and game ended in draw

    # MCTS uses Max and Min players
    MAX = X # Player Max is the X token (as X plays first)
    MIN = O


    def __init__(self, n):
        """
            :param n: Dimensions of the board - Since the board is a square board, the board will be an nxn board

            k is the number of tokens needed in a row to win
            neutral_tokens is the number of neutral tokens ('S') that the board will contain
        """
        self.n = n

        self.neutral_tokens = 0

        if self.n == 4:
            self.k = 3
            self.neutral_tokens = 1
        elif self.n in [5, 6]:
            self.k = 4
            self.neutral_tokens = 2
        elif self.n == 7:
            self.k = 5
            self.neutral_tokens = 3
        else:
            pass  # We've already covered all cases and validation of n will be done when getting input from the user



    def check_in_diagonal(self, board, row, col):
        """
            Check if there are 2 neutral tokens in the same diagonal

            :return: True if so else False 
        """
        temp_board = [[0 for i in range(self.n + 4)] for j in range(self.n + 4)]

        for i in range(self.n):
            for j in range(self.n):
                temp_board[i + 2][j + 2] = board[i][j]

        if (temp_board[row - 2 + 2][col - 2 + 2] == self.S or temp_board[row - 1 + 2][col - 1 + 2] == self.S or
                temp_board[row + 2 + 2][col + 2 + 2] == self.S or temp_board[row + 1 + 2][col + 1 + 2] == self.S or
                temp_board[row + 2 + 2][col - 2 + 2] == self.S or temp_board[row + 1 + 2][col - 1 + 2] == self.S or
                temp_board[row - 2 + 2][col + 2 + 2] == self.S or temp_board[row - 1 + 2][col + 1 + 2] == self.S):
            return True

        else:
            return False


    def print_2D_array(self,array):
        for i in range(len(array)):
            output = ''
            for j in range(len(array[i])):
                output = output + (str(array[i][j])) + ' '
            print(output)



    def initial_game_board(self):
        """
            Generate an initial board for the game with the neutral tokens (randomly) placed
             on it, and return it

            :return: An initial board of the game (the initial game state) as a 2D array
        """

        # TO IMPLEMENT - ensure no neutral tokens are on the same row, column, or diagonal as any other neutral tokens

        board = [[self.EMPTY for i in range(self.n)] for j in range(self.n)]  # initialise a nxn board using Python List Comprehension

        neutral_rows = []
        neutral_cols = []
        rand_row = 0
        rand_col = 0

        for m in range(self.neutral_tokens):
            randRow = random.randint(0, self.n - 1)  # For randint(a, b) both bounds are inclusive i.e. range is [a, b]
            randCol = random.randint(0, self.n - 1)

            # While the coordinates of this square is not valid for a neutral token, generate new coordinates
            # The condition of the while loop also checks if the current square position already has a neutral token placed on it
            while (self.check_in_diagonal(board, rand_row, rand_col) or (rand_row in neutral_rows) or (rand_col in neutral_cols)):
                rand_row = random.randint(0, self.n - 1)
                rand_col = random.randint(0, self.n - 1)

            board[rand_row][rand_col] =  self.S
            neutral_rows.append(rand_row)
            neutral_cols.append(rand_col)

        '''
        token = 0

        #Azhar's code
        check = False # flag for checking neutral tokens
        while not check:
            neutral_squares = list() # list of neutral token co-ordinates

            while token < self.neutral_tokens:
                randRow = random.randint(0, self.n-1) # For randint(a, b) both bounds are inclusive i.e. range is [a, b]
                randCol = random.randint(0, self.n-1)

                if board[randRow][randCol] != self.S: # ALT: board[randRow][randCol] == self.EMPTY
                    # Do check here if square is a valid place for a neutral token, and if it is then do the code below
                    board[randRow][randCol] = self.S
                    neutral_squares.append((randRow,randCol))    # (,) is a tuple                
                    token = token + 1

            #Azhar's code
            """
            if abs(neutral_squares[0][0]-neutral_values[0][1])!=abs(neutral_values[1][0]-neutral_values[1][1])!= # Check Diagonals abs(X,Y)
            abs(neutral_values[2][0]-neutral_values[2][1]) or neutral_values[0][0] != neutral_values[1][0] != # Check row
            != neutral_values[2][0] or neutral_values[0][1] != neutral_values[1][1] != neutral_values[2][1]:  # Check col
                check = True
            else:
                token = 0
            """
            '''
        self.print_2D_array(board)
        return board



    def create_game_state(self, neutral_moves=None, players_moves=None):
        """
            Create a game situation based on the actions given

            :neutral_moves: a list of the squares on the board that the neutral tokens are placed on - each element
                is a tuple of size 2 indicating the position of the square.  the length of this list must be equal to self.neutral_tokens.
                If None then call self.initial_game_board() to randomly place the neutral tokens
            :players_moves: moves that have already been played by both players - a list of tuples. Each tuple is of size 3.
                For each tuple, the first element is the player whose turn it is (either X or Y), second element is the x coordinate of the square,
                and the third element is the y coordinate. Ensure the coordinates are valid board positions
            :return: a State of the game with its board populated with the specified actions/moved
        """

        turn = self.X


        if neutral_moves == None:
            board = self.initial_game_board()
        else:
            board = [[self.EMPTY for i in range(self.n)] for j in range(self.n)]  # initialise a nxn board using Python List Comprehension

            # Add neutral moves from neutral_moves
            for i in range(len(neutral_moves)):
                board[neutral_moves[i][0]][neutral_moves[i][1]] = self.S  # neutral_moves[i][0] is x-coordinate of that tuple and neutral_moves[i][1] is y-coordinate of that tuple

        # Add the moves of the players
        if players_moves != None:
            for i in range(len(players_moves)):
                player = players_moves[i][0]
                x_coord = players_moves[i][1]
                y_coord = players_moves[i][2]
                board[x_coord][y_coord] = player

            """code to determine the player whose turn it is to play next on this current state"""
            num_x = (np.array(board) == self.X).sum()
            num_y = (np.array(board) == self.O).sum()

            if num_x > num_y:
                turn = self.O
            else:
                turn = self.X

        return State(board, turn)


    def moves(self, state: State):
        """
            Determine all the actions/moves (x,y pairs) that can be played on the current board state
            and return them

            A move is an empty square that the current player can place their token on
            (It doesn't matter who the current player is)

            :return: a list of tuples, with each tuple being the position/coordinates of a square on the board
        """

        board = state.board

        # A list of tuples
        moves = list()
        board = state.board

        for i in range(len(board)):
            for j in range(len(board[i])):
                # Check if the cell is empty
                if board[i][j] == self.EMPTY:
                    # Create a tuple holding a possible i, j move
                    move_tuple = (i, j)
                    # Add the tuple to the list
                    moves.append(move_tuple)

        return moves


    def play_move(self, state: State, move):
        """
            Play the specified action/move on this current state (resulting in a child state)
            :return: a new State with the move having being played on it
        """
        # Assumption: move is a tuple
        # Play the move on this state
        board = state.board
        child_board = copy.deepcopy(board)
        child_board[move[0]][move[1]] = state.turn

        # Return a new state - updated board - updated turn
        return State(board=child_board, turn=-state.turn)  # turn=-self.turn since TicTacToeGame.X == -TicTacToeGame.O and vice versa


    def get_child_states(self, state: State):
        """
            Generate and return all child states of this state with the current player being state.turn

            Use this method for expansion

            :return: a list of all child States of this state
        """

        child_moves = self.moves(state)
        board = state.board
        child_states = list()

        for move in child_moves:  # move is a tuple represeting the position/coordinates of a square on the board
            child_board = copy.deepcopy(board)
            i = move[0]
            j = move[1]
            child_board[i][j] = state.turn
            child_states.append(State(board=child_board, turn=-state.turn))  # toggle/switch the current player

        """
        Deprecated
        for i in range(len(board)):
            for j in range(len(board[i])):
                if (board[i][j] == TicTacToeGame.EMPTY):
                    child_board = copy.deepcopy(board)
                    child_board[i][j] = state.turn
                    child_states.append(State(board=child_board, turn=-state.turn)) # toggle/switch the current player
        """

        return child_states


    def get_empty_squares(self, state: State):
        """
            :return: a list of tuples, with each tuple representing the coordinates of an empty square on the board
        """

        board = state.board
        empty_squares = list()

        for i in range(len(board)):
            for j in range(len(board[i])):
                if state.board[i][j] == self.EMPTY:
                    empty_square = (i, j)  # (i, j) is a tuple
                    empty_squares.append(empty_square)

        return empty_squares


    def display_state(self, state: State):
        """
            A function for display the board state
        """

        board = state.board

        for i in range(len(board)):
            line = ""
            for j in range(len(board[i])):
                character = board[i][j]
                if character == self.EMPTY:
                    character = '_'
                elif character == self.X:
                    character = 'X'
                elif character == self.O:
                    character = 'O'
                else:
                    character = 'S'
                line += "|" + str(character)
            line += "|"
            print(line)


    def check_game_over(self, state: State):
        """
            Determines whether the game is over or not for this state and 
            returns whether the game is over or not, the winner (or lack of one)
            and the utilty (score/value) of this (terminal) state

            The game is over if either X won, O won, or game is a draw
            If the game is over, then this state is a terminal state

            Utility value is 1 if X won, -1 if O won, 0.5 if there is a draw, 
            or 0 if this state is not a terminal state (the game is currently in progress)

            :return: a (is_terminal_state, winner, utilty) tuple 
        """

        # Default - game is not over - currently in progress 
        is_terminal_state = False
        winner = self.IN_PROGRESS 
        utilty = 0

        board = state.board

        # ***** TO IMPLEMENT *****
        # If the state represents a game over scenario, set the is_terminal_state to True and the winner variable to either self.X, self.O, or self.DRAW
        # to check if there's no empty square left (for a draw condition), can use the following code: np.array(self.board == self.EMPTY)).sum() == 0

        #...

        if is_terminal_state:
            if winner == self.DRAW:
                utility = 0.5
            else:
                utility = winner # TicTacToeGame.X is represented by 1 and TicTacToeGame.O is represented by -1 so the corresponding value for the winner will be returned

        return is_terminal_state, winner, utility


    def check_win(self, state: State):
        for i in range(len(state.board)):
            for j in range(len(state.board[i])):
                if (self.check_vertical_k_win(state.board, i, j, state.turn, self.k, self.n, self.n) or 
                    self.check_horizontal_k_win(state.board, i, j, state.turn, self.k,self.n,self.n) or 
                    self.check_positive_diagonal_k_win(state.board, i, j, state.turn, self.k, self.n, self.n) or 
                    self.check_negative_diagonal_k_win(state.board, i, j, state.turn, self.k, self.n, self.n)):
                    return True
        return False


    def check_vertical_k_win(self, matrix, row, col, token, k, rows, cols):
        n = int(k / 2)
        tempMatrix = [[0 for i in range(cols + (n * 2))] for j in range(rows + (n * 2))]
        for i in range(rows):
            for j in range(rows):
                tempMatrix[i + n][j + n] = matrix[i][j]
        count = 0
        countS = 0
        sArray = []
        sArray.append(token)

        if row <= rows-2:
            for l in range(k):
                if ((tempMatrix[row + n + l][col + n] == token) or (tempMatrix[row + n + l][col + n] == TicTacToeGame.S)):
                    count = count + 1
                elif (token == TicTacToeGame.S):
                    if (len(sArray) == 1):
                        sArray.append(tempMatrix[row + n + l][col + n])
                    if (tempMatrix[row + n + l][col + n] in sArray):
                        countS = countS + 1
                    else:
                        countS = 0

        if (count == k or countS == k):
                return True
        return False


    def check_horizontal_k_win(self, matrix, row, col, token, k, rows, cols):
        n = int(k / 2)
        tempMatrix = [[0 for i in range(cols + (n * 2))] for j in range(rows + (n * 2))]
        for i in range(rows):
            for j in range(rows):
                tempMatrix[i + n][j + n] = matrix[i][j]
        count = 0
        countS = 0
        sArray = []
        sArray.append(token)

        if col <= cols-2:
            for l in range(k):
                if ((tempMatrix[row + n][col + n + l] == token) or (tempMatrix[row + n][col + n + l] == TicTacToeGame.S)):
                    count = count + 1
                elif (token == TicTacToeGame.S):
                    if (len(sArray) == 1):
                        sArray.append(tempMatrix[row + n][col + n + l])
                    if (tempMatrix[row + n][col + n + l] in sArray):
                        countS = countS + 1
                    else:
                        countS = 0

        if (count == k or countS == k):
                    return True
        return False


    def check_negative_diagonal_k_win(self, matrix, row, col, token, k, rows, cols):
        n = int(k / 2)
        tempMatrix = [[0 for i in range(cols + (n * 2))] for j in range(rows + (n * 2))]
        for i in range(rows):
            for j in range(rows):
                tempMatrix[i + n][j + n] = matrix[i][j]
        count = 0
        countS = 0
        sArray = []
        sArray.append(token)

        if row <= rows - 2 and col <= cols-2:
            for l in range(k):
                if ((tempMatrix[row + n + l][col + n + l] == token) or (tempMatrix[row + n + l][col + n + l] == TicTacToeGame.S)):
                    count = count + 1
                elif (token == TicTacToeGame.S):
                    if (len(sArray) == 1):
                        sArray.append(tempMatrix[row + n + l][col + n + l])
                    if (tempMatrix[row + n + l][col + n + l] in sArray):
                        countS = countS + 1
                    else:
                        countS = 0

        if (count == k or countS == k):
                    return True
        return False


    def check_positive_diagonal_k_win(self, matrix, row, col, token, k, rows, cols):
        n = int(k / 2)
        tempMatrix = [[0 for i in range(cols + (n * 2))] for j in range(rows + (n * 2))]
        for i in range(rows):
            for j in range(rows):
                tempMatrix[i + n][j + n] = matrix[i][j]
        count = 0
        countS = 0
        sArray = []
        sArray.append(token)

        if row <= rows - 2 and col <= cols-2:
            for l in range(k):
                print(tempMatrix[row + n - l][col + n + l])
                if ((tempMatrix[row + n - l][col + n + l] == token) or (tempMatrix[row + n - l][col + n + l] == TicTacToeGame.S)):
                    count = count + 1
                elif (token == TicTacToeGame.S):
                    if (len(sArray) == 1):
                        sArray.append(tempMatrix[row + n - l][col + n + l])
                    if (tempMatrix[row + n - l][col + n + l] in sArray):
                        countS = countS + 1
                    else:
                        countS = 0

        if (count == k or countS == k):
                    return True
        print(count)
        print(countS)
        return False





# Node Class

[The root node of the tree is the current game state on which we want to decide the best move to make]


Node class for adversarial game: 

Option 1 (this is what sir mentions in notes): have 2 separate win counters, one for player 1 (Max) and one for player 2 (Min) and the one that is used to calculate UCT value is the player who's turn it is to currently play (Me: in state 0 - the initial state of the algo - the current state of the game). So if terminal value is positive then increment Player 1's score, else if negative then increment Player 2's score with the absolutve value of the terminal value. (We can also increment both players' scores (with half the value of a win - if win=1 then draw =0.5 for both players) if there is a draw). 
[AIMA mentions about having a vector of values per node for multiplayer games]

Option 2: have 1 score value, higher value indiciating preference to Max and lower value indicating preference to Min (thus value can go negative). So algo selects the node with either the best or worst UCT value depending on who's turn it is

Option 3 (See: https://en.wikipedia.org/wiki/Monte_Carlo_tree_search#Principle_of_operation): have 1 score/value counter for each node, and that score reperesents the player that that node represents. So when backpropagating, we update every alternate ascendant node with the score (the start of the alternation depends on whether the leaf node that was rolled out resulted in a win/loss for the player that that lead node represents). (So if selecting a child node, then the current node will have to choose the child with the minimum value as the values of the children represent that of the opposite player)
[This option is similar to Option 1]


Going with Option 1

Modelling: use MaxV and MinV for the scores of the 2 players in the Node representation. Max is the X token (the player who plays the first move of the game) and Min is the O token


calculate_uct_value_() explanation (also see its docstring):

in select_best__child() for the current node, we calculating the uct of each of its child nodes and selecting the one that has the highest uct value.
But the node that has the highest uct value needs to be in terms of this node - the player of this current node's state whose turn it is to make a move. cos we want to select the best move that this current player should play.

So when calculating the child node that has the highest uct value, it needs to be in terms of the player of this current node.

say the current node is X's turn to play. we want to select a child node of X (all of which are O's turn to play) that is the best move for x - so we want to get the node that has the max wins for X in the rollouts

all of the child nodes are O's turns. so when calcuating uct of the child node, we use the player of the parent node, which is X. and vice versa if current node is O's turn to play


In [ ]:
class Node:
    """
        A node for the Monte Carlo Tree Search algorithm, that is based on 
        a Tic Tac Toe State
     """
 
 
    def __init__(self, state: State, parent=None):
        """
            Constructor for the Node class

            Parameters:
            :param state (State): An instance of the state of the game
            :param parent (Node): The parent node of the current node
        """

        # The number of wins for player Max through this node
        self.Max_V = 0

        # The number of wins for player Min through this node
        self.Min_V = 0

        # The number of times the node (or its children) has been rolled out
        self.n = 0

        # The parent Node
        self.parent = parent

        # The Game State
        self.state = state

        # Chilren Nodes
        self.children = list()
    

    def calculate_uct_value(self):
        """
            Calculate and return the UCT value of this node based on the player whose turn it is 
            in the state of the parent node of this node

            We are using the player of the parent node's state instead of the player
            of this node's state as this node's state is a result of the player
            of the parent node's state making a move, and the UCT value will be used
            to determine the quality of the move from the parent nodes perspective.

            But if the current node is the root node of the tree (it has no parent)
            then we'll use the player of this root node's state. Though this function won't
            be called on the root node of a tree as it called on children nodes of a node. 
            But this case is placed here for validation purposes so that there's no error
            if parent=None
        
            :return: uct: Float
        """

       
        if self.n == 0:  # Check for 0 division
            # Return positive infinity (This node must be set to a high priority for Exploration)
            return np.inf
        else:
            v = 0

            if parent != None:
                if parent.state.turn == TicTacToe.X:
                    v = self.Max_V # Player Max is X
                else:
                    v = self.Min_V # Player Min is Y
            else: # if root node of the tree - it doesn't have a parent node
                if self.state.turn == TicTacToe.X:
                    v = self.Max_V
                else:
                    v = self.Min_V 

            # Return uct value for this node (based on its parent node's state's current player)
            return (v/self.n) + C * math.sqrt(np.log(self.parent.n)/self.n)



    def select_best_child(self):
        """
            Determine the best child node of this node

            :return: best_child: Node
        """
        max_uct = 0
        best_child = None
        for child in self.children:
            uct_value = child.calculate_uct_value()
            if uct_value > max_uct:
                max_uct = uct_value
                best_child = child
        return best_child


    def to_string(self):
        output = "Node Summary\nNumber of wins through this node: {}\nNumber of visits: {}\nUCT value = {}".format(self.v, self.n, self.calculate_uct_value())
        return output

# Monte Carlo Tree Search

AIMA: When the iterations terminate, the move with the highest number of Playouts/Rollouts is returned. You might think that it would be better to return the node with the highest average utility, but the idea is that a node with 65/100 wins is better than one with 2/3 wins, because the latter has a lot of uncertainty. In any event, the UCB1 formula ensures that the node with the most playouts is almost always the node with the highest win percentage, because the selection process favors win percentage more and more as the number of playouts goes up.


Todo - what happens if a node we at represents a terminal state and the algorithm wants to generate its child nodes


In [ ]:
def monte_carlo_tree_search(game, state, iterations=100):
    """
        Conduct the MCTS from the given state (the state that is passed in)
        Return the best move to make on the state passed in (the current state of the game)
        along with its resulting state
    """

    # Child/Sub Functions

    def add_child_nodes(node):
        """
            Create children nodes of this node and add it to the node as its children
        """

        state = node.state
        child_states = game.get_child_states(state)
        node.children = list() # initialise to an empty list
        
        if child_states: # if the child_states list is not empty
            for child_state in child_states:
                node.children.append(Node(child_state, parent=node)) # add child state as a child node to this current node


    def random_action(state):
        """
            Randomly select an action/move from all the possible moves that
            can be made on this state and return it

            :returns: random_move: A tuple (i, j) to place a move
        """

        possible_moves = game.moves(state)
        option = random.randint(0, len(possible_moves)-1)
        random_move = possible_moves[option]
        return random_move


    def simulate(state, move):
        """
            Apply a random action on the state

            Return:
            new_state: State
        """

        new_state = game.play_move(state, move)
        return new_state


    def rollout(state):

        while True:
            
            is_terminal_state, winner, utility = game.check_game_over(state)
            
            if is_terminal:
                return winner, utility

            action = random_action(state)
            state = simulate(state, action)


    def backpropagate(node, winner, result):
        """
            :param node: the node that rollout was done on and to start backpropagating
                the result from
            :param winner: the winner value returned by check_game_over()
            :param result: the utility value returned by check_game_over()
        """

        while node.parent != None:
            
            node.n += 1

            if winner == game.DRAW:
                node.Max_V += result
                node.Min_V += result
            elif winner == game.MAX:
                node.Max_V += result
            else: # if winner == game.MIN
                node.Min_V += result

            node = node.parent


    # MCTS - still working on it

    current = Node(state)
    count = 0
  
    while count < iterations:
        current = Node(state) # reset current for this new iteration

        """
            While current is not a leaf node (has children nodes), get the best child of current.
            Stop when current is a leaf node (doesn't have any children)

            current.children will evaluate to true if the children list is
            not empty (current has children nodes), and will evaluate to false
            if the children list is empty (current doesn't have any children
            nodes -> current is a leaf node)

        """
        while current.children: # Tree Traversal: while current is not a leaf node
            current = current.select_best_child()

        # current is now a leaf node (doesn't have any children)
        is_terminal_state, winner, utility = game.check_game_over(current.state)
        
        # (VERIFY THIS PART) if the state of the current node is a terminal state, then return its utilty
        # if current.n == 0  then its utility will be returned anyway in rollout()
        # but if current.n != 0 there will be an issue as this node will not have any child nodes
        # and we'll be accessing the first child node of this node
        if is_terminal_state:
            winner, result = winner, utility
        else: # if the state of the current (leaf) node is not a terminal state 
            if current.n == 0:
                winner, result = rollout(current.state)
            else:
                add_child_nodes(current)
                #current = current.select_best_child()
                current = current.children[0] # First new child node
                winner, result = rollout(current.state)

        backpropagate(current, winner, result)

    # Return best action/move to make



# Examples


In [ ]:

# Dimension of the board from user
try:
  size = int(input("Enter the size of the square board (4,5,6,7): "))

  if size < 4 or size > 7:
    size = 4 # Default if invalid value entered
except ValueError:
  size = 4 # Default if invalid (non-int) input

# Create game instance
ttt_game = TicTacToeGame(size)

ttt_game.initial_game_board()

"""
Create an initial state
No X's or O's are present
"""

initial_state = ttt_game.create_game_state()
next_state = ttt_game.get_child_states(initial_state)
new_state = next_state[5]
move = [2,1]
next_state = ttt_game.play_move(new_state,move)

ttt_game.display_state(next_state)


monte_carlo_tree_search(ttt_game, next_state, 100)

"""
print((np.array(initial_state.board)==2).sum())

current_state = initial_state

complete_game_tree = monte_carlo_tree_search(ttt_game, current_state, iterations=200)
while not complete_game_tree.parent == None:
    complete_game_tree.state.display_state()


print("Board Size: {}x{}\nNumber of tokens needed to win: {}\nNumber of Neutral Tokens available: {}".format(TIC_TAC_TOE_GAME.n, TIC_TAC_TOE_GAME.n, TIC_TAC_TOE_GAME.k, TIC_TAC_TOE_GAME.neutral_tokens))"""

In [17]:
X = TicTacToeGame.X
O = TicTacToeGame.O

four_x_four_1 = TicTacToeGame(4)
four_x_four_State1 = four_x_four_1.create_game_state([(0,0)],[(X,3,0),(O,3,3),(X,1,2)])
four_x_four_1.display_state(four_x_four_State1)

print("")

# FIX THE FOLLOWING GAMES - SET THE PLAYER PARAM AND MAKE THE MOVE BRACKETS BE ROUND BRACKETS

four_x_four_2 = TicTacToeGame(4)
four_x_four_State2 = four_x_four_2.create_game_state([[1,1]],[[3,0],[3,2],[0,3],[3,3]])
four_x_four_2.display_state(four_x_four_State2)

print("")

five_x_five_1 = TicTacToeGame(5)
five_x_five_State1 = five_x_five_1.create_game_state([[3,0],[1,4]],[[2,2],[3,3]])
five_x_five_1.display_state(five_x_five_State1)

print("")

six_x_six_1 = TicTacToeGame(6)
six_x_six_State1 = six_x_six_1.create_game_state([[1,1],[4,4]],[[0,0],[2,2],[3,3],[5,5]])
six_x_six_1.display_state(six_x_six_State1)

print("")

seven_x_seven_1 = TicTacToeGame(7)
seven_x_seven_State1 = seven_x_seven1.create_game_state([[6,1],[2,2],[3,5]],[[6,2],[1,3],[3,4]])
seven_x_seven_1.display_state(seven_x_seven_State1)

print("")

seven_x_seven_2 = TicTacToeGame(7)
seven_x_seven_State2 = seven_x_seven_2.create_game_state([[6,1],[2,2],[4,5]],[[3,1],[2,0],[1,2],[1,3],[3,4]])
seven_x_seven_2.display_state(seven_x_seven_State2)

|S|_|_|_|
|_|_|X|_|
|_|_|_|_|
|X|_|_|O|



IndexError: ignored